In [1]:
# Use this when core llama-index functions do not work
# !pip install llama-index --upgrade --no-cache-dir --force-reinstall

In [2]:
# %pip install llama-index-embeddings-openai
# %pip install llama-index-postprocessor-cohere-rerank
# %pip install llama-index-llms-openai

In [3]:
# !pip install llama-index-llms-ollama

In [4]:
import os
from llama_index.core import Settings



### 1. Initialize LLM 

In [5]:
from llama_index.llms.ollama import Ollama
# from llama_index.llms.openai import OpenAI

In [7]:
llm = Ollama(model="nous-hermes2", 
             request_timeout=30.0)

resp = llm.complete("Who are you? What is your name?")
print(resp)

My name is AI Assistant. I am here to help you with any questions or information you need. How can I assist you today?


### 2. Chunking the text from files

In [26]:
# !pip install llama-index-vector-stores-chroma

In [36]:
# load some documents
documents = SimpleDirectoryReader("./data").load_data()
print(len(documents))

1


In [37]:
## Optional for joining PDF pages if each page is treated as a seperate document


# def concatenate_documents(documents):
#     # Step 1: Create a list of unique file names
#     unique_file_names = set(doc.metadata['file_name'] for doc in documents)

#     # Step 2-5: Concatenate documents with the same file name
#     concatenated_documents = []
#     for file_name in unique_file_names:
#         concatenated_text = ''
#         initialized_doc = None
#         for doc in documents:
#             if doc.metadata['file_name'] == file_name:
#                 if initialized_doc is None:
#                     initialized_doc = doc
#                 else:
#                     initialized_doc.text += doc.text
#         concatenated_documents.append(initialized_doc)

#     return concatenated_documents


# documents = concatenate_documents(documents_simple)

In [44]:
from llama_index.core.node_parser import SentenceSplitter
node_parser = SentenceSplitter(chunk_size=1000, chunk_overlap = 200)#factsheet_textract_1000

#node_parser = TokenTextSplitter()
nodes = node_parser.get_nodes_from_documents(documents)

print(len(nodes))
nodes[0]

4


TextNode(id_='55c89e74-de07-469d-a3f8-0adc85a5b9e3', embedding=None, metadata={'file_path': 'c:\\Users\\vibud\\Desktop\\Github\\TensorFlow-for-DeepLearning\\K-llamaIndex\\v0.10\\data\\HelloTest.txt', 'file_name': 'HelloTest.txt', 'file_type': 'text/plain', 'file_size': 14608, 'creation_date': '2024-03-27', 'last_modified_date': '2024-03-27'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3c3eafa1-da8b-43ff-a062-fdb7ddd0260c', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'c:\\Users\\vibud\\Desktop\\Github\\TensorFlow-for-DeepLearning\\K-llamaIndex\\v0.10\\data\\HelloTest.txt', 'file_name': 'HelloTest.txt', 'file_type': 'text/plain', 'file_size': 14608, 'creation_date': '2024-03-27', 'last_m

In [ ]:
# def extract_date_from_text(text: str) -> str:
#     date_pattern = r'\b(AS OF ?:JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER) \d{1,2}, \d{4}\b'
#     match = re.search(date_pattern, text)
#     try:
#         if match:
#             date_string = match.group(0)
#             parsed_date = dateutil.parser.parse(date_string)
#             formatted_date = parsed_date.strftime("%Y%m%d")
#             print(formatted_date)
#             return formatted_date
#         else:
#             date_pattern = r'\b(AS OF ?:January|February|March|April|May|June|July|August|September|October|November|December) \d{4}\b'
#             date_pattern = r'\b(AS OF ?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}\b'
#             match = re.search(date_pattern, text)
#             date_string = match.group(0)
#             parsed_date = dateutil.parser.parse(date_string)
#             formatted_date = parsed_date.strftime("%Y%m%d")
#             #last_day = last_day_of_month(int(str(formatted_date)[0:4]), int(str(formatted_date)[4:]))
#             #formatted_date = str(formatted_date)+str(last_day)
#             print(formatted_date)
#             return formatted_date
#     except Exception as e:
#         return '19990101'


# for i, node in enumerate(nodes):
#     """Metadata:
#     doc_name
#     Company_Name
#     DocType: Ammendment 1 or Original Contract
#     chunk
#     index
#     """
#     doc_name = node.metadata['file_name'].split(".")[0]
#     #node.excluded_llm_metadata_keys=('chunk')
#     if "LukkaAmd1" in doc_name:
#         node.metadata["Doc_Name"] = doc_name
#         node.metadata["Company_Name"] = "Lukka, Inc."
#         node.metadata["DocType"] = "Ammendment 1"
#         node.metadata['chunk'] = i+1
#         node.metadata['index'] = node.metadata['file_name'].split(".")[0]
        
#     elif "Lukka_SPDJI_Data Master" in doc_name:
#         node.metadata["Doc_Name"] = doc_name
#         node.metadata["Company_Name"] = "Lukka, Inc."
#         node.metadata["DocType"] = "Original Contract"
#         node.metadata['chunk'] = i+1
#         node.metadata['index'] = node.metadata['file_name'].split(".")[0]

#     # node.metadata['file_type'] = 'Finance'
#     # node.metadata['chunk'] = i+1
#     # node.metadata['index'] = node.metadata['file_name'].split(".")[0]
#     # node.metadata['doc_name'] = node.metadata['file_name'].split(".")[0]
#     #node.metadata['version'] = extract_date_from_text(node.text)
#     #node.excluded_llm_metadata_keys=('chunk')

### 3. Creating vectorStore 

In [46]:
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
# from llama_index.index_store.storage_context import StorageContext

In [48]:
# load some documents
documents = SimpleDirectoryReader("./data").load_data()

# create Chroma vector store
chroma_client = chromadb.Client()
chroma_store = ChromaVectorStore(chroma_client)

# create index using Chroma vector store
index = VectorStoreIndex.from_documents(documents, vector_store=chroma_store)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\vibud\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\vibud\AppData\Local\Temp\ipykernel_16532\1138279434.py", line 5, in <module>
    chroma_client = chromadb.Client()
                    ^^^^^^^^^^^^^^^^^
  File "c:\Users\vibud\miniconda3\Lib\site-packages\chromadb\__init__.py", line 274, in Client
    return ClientCreator(tenant=tenant, database=database, settings=settings)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vibud\miniconda3\Lib\site-packages\chromadb\api\client.py", line 143, in __init__
    self._admin_client = AdminClient.from_system(self._system)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vibud\miniconda3\Lib\site-packages\chromadb\api\client.py", line 495, in from_system
    instance = cls(settings=system.settings)
        

In [42]:
# create Chroma vector store
chroma_client = chromadb.Client()
chroma_store = ChromaVectorStore(chroma_client)

# create index using Chroma vector store
index = VectorStoreIndex.from_documents(documents, vector_store=chroma_store)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\vibud\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\vibud\AppData\Local\Temp\ipykernel_16532\2167038500.py", line 2, in <module>
    chroma_client = chromadb.Client()
                    ^^^^^^^^^^^^^^^^^
  File "c:\Users\vibud\miniconda3\Lib\site-packages\chromadb\__init__.py", line 274, in Client
    return ClientCreator(tenant=tenant, database=database, settings=settings)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vibud\miniconda3\Lib\site-packages\chromadb\api\client.py", line 143, in __init__
    self._admin_client = AdminClient.from_system(self._system)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vibud\miniconda3\Lib\site-packages\chromadb\api\client.py", line 495, in from_system
    instance = cls(settings=system.settings)
        

In [35]:
import chromadb 
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("10-K")

AttributeError: 'Settings' object has no attribute 'chroma_segment_cache_policy'

In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.embeddings.openai import OpenAIEmbedding

# Load the BGE-small-en-v1.5 model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [9]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")

chroma_collection = db2.get_or_create_collection("DB_collection")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

NameError: name 'chromadb' is not defined

In [10]:
from llama_index.core import SimpleDirectoryReader

document = SimpleDirectoryReader("./data").load_data()

In [11]:
print(len(document))
print(type(document))

1
<class 'list'>


In [12]:
import os
from llama_index.core import (
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)

if not os.path.exists("storage"):
    index = VectorStoreIndex.from_documents(docs)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage")
else:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage")
    # load index
    index = load_index_from_storage(storage_context, index_id="vector_index")

In [13]:
from llama_index.core.query_pipeline import QueryPipeline
from llama_index.core import PromptTemplate

# try chaining basic prompts
prompt_str = "Please generate packages  related to {disease_name}"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model="local-model",
            temperature=0.7, 
            max_tokens=256)

p = QueryPipeline(chain=[prompt_tmpl, llm], verbose=True)

ValueError: Unknown model 'local-model'. Please provide a valid OpenAI model name in: gpt-4, gpt-4-32k, gpt-4-1106-preview, gpt-4-0125-preview, gpt-4-turbo-preview, gpt-4-vision-preview, gpt-4-0613, gpt-4-32k-0613, gpt-4-0314, gpt-4-32k-0314, gpt-3.5-turbo, gpt-3.5-turbo-16k, gpt-3.5-turbo-0125, gpt-3.5-turbo-1106, gpt-3.5-turbo-0613, gpt-3.5-turbo-16k-0613, gpt-3.5-turbo-0301, text-davinci-003, text-davinci-002, gpt-3.5-turbo-instruct, text-ada-001, text-babbage-001, text-curie-001, ada, babbage, curie, davinci, gpt-35-turbo-16k, gpt-35-turbo, gpt-35-turbo-0125, gpt-35-turbo-1106, gpt-35-turbo-0613, gpt-35-turbo-16k-0613

In [ ]:
output = p.run(disease_name="Call back Manager")

> Running module 6d204937-c259-41b3-8ae8-6bec0f71bfe6 with input: 
disease_name: Call back Manager

> Running module 65e68ba9-a0c8-4461-bc92-c0a073ab3056 with input: 
messages: Please generate packages  related to Call back Manager



Retrying llama_index.llms.openai.base.OpenAI._chat in 0.04620972861039607 seconds as it raised InternalServerError: Error code: 500 - {'error': "Model with key 'gpt-3.5-turbo' not loaded."}.
Retrying llama_index.llms.openai.base.OpenAI._chat in 1.54540937889328 seconds as it raised InternalServerError: Error code: 500 - {'error': "Model with key 'gpt-3.5-turbo' not loaded."}.
Retrying llama_index.llms.openai.base.OpenAI._chat in 3.5407211030882673 seconds as it raised InternalServerError: Error code: 500 - {'error': "Model with key 'gpt-3.5-turbo' not loaded."}.
Retrying llama_index.llms.openai.base.OpenAI._chat in 0.35979374943517595 seconds as it raised InternalServerError: Error code: 500 - {'error': "Model with key 'gpt-3.5-turbo' not loaded."}.
Retrying llama_index.llms.openai.base.OpenAI._chat in 3.2271254198093438 seconds as it raised InternalServerError: Error code: 500 - {'error': "Model with key 'gpt-3.5-turbo' not loaded."}.


InternalServerError: Error code: 500 - {'error': "Model with key 'gpt-3.5-turbo' not loaded."}

In [ ]:
# # Set the global service context to use this embedding model
# from llama_index import ServiceContext, set_global_service_context
# service_context = ServiceContext.from_defaults(embed_model=embed_model)
# set_global_service_context(service_context)

In [ ]:
# os.environ["OPENAI_API_KEY"] = "Not Needed"
# os.environ["OPENAI_API_BASE"] = "http://localhost:1234/v1"

# llm = OpenAI(temperature=0.7, max_tokens=256)

In [ ]:
# Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")